In [13]:
import pandas as pd
import json
import os
from selectolax.parser import HTMLParser
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import glob

from datetime import date

In [14]:
today = date.today()

In [15]:
input_24 = pd.read_excel(f'Output/{str(today)}_esiti_positivi_2500_v1.xlsx')

In [16]:
input_24= input_24[input_24['stato']=='IN ATTESA DI PAGAMENTO']
#  input_24= input_24[input_24['compila']=='si']

In [17]:
def apply_mapping(value):
    if 'DECRETO INGIUNTIVO' in value:
        return 'DI - DECRETO INGIUNTIVO'
    elif 'ESECUZIONE MOBILIARE' in value:
        return 'EM - ESECUZIONE MOBILIARE'
    elif 'ORDINANZA' in value:
        return 'OR - ORDINANZA'
    elif 'ATTO VARIO' in value:
        return 'AV - ATTO VARIO'
    else:
        return None  # Default value if no match is found

def apply_mapping_tipo(value):
    if 'DI - DECRETO INGIUNTIVO' in value:
        return 'DECRETO'
    elif 'EM - ESECUZIONE MOBILIARE' in value:
        return 'PPT'
    elif 'OR - ORDINANZA' in value:
        return 'PPT'
    elif 'ATTO VARIO' in value:
        return 'DECRETO'
    else:
        return None    

# Apply the mapping to create the 'natura' column
input_24['natura'] = input_24['atto'].apply(apply_mapping)
input_24['tipo'] = input_24['natura'].apply(apply_mapping_tipo)

In [18]:
def extract_numero(value):
    numero_match = re.search(r'n\. (\d+)', value)
    if numero_match:
        return numero_match.group(1)
    else:
        return None

def extract_anno(value):
    anno_match = re.search(r'del (\d+)', value)
    if anno_match:
        return anno_match.group(1)
    else:
        return None

# Apply the functions to create the 'numero' and 'anno' columns
input_24['numero'] = input_24['atto'].apply(extract_numero)
input_24['anno'] = input_24['atto'].apply(extract_anno)

In [19]:
"""
# aggiungiamo il nome del cliente 
clienti = pd.read_excel('Input/Raw/perimetro_27_05.xlsx')
clienti = clienti[['NR_Contratto',"Cliente"]]
clienti['Cliente'] = clienti['Cliente'].str.lower()
input_24 = pd.merge(input_24, clienti, left_on='contratto',right_on='NR_Contratto', how='left')
input_24.drop(columns=['NR_Contratto'], inplace=True)
"""

In [20]:
# aggiungiamo la p.iva/cf del cliente e denominazione (?)
cf_piva_clienti = pd.read_excel('Resources/cf_piva_f24.xlsx', dtype={'CF': str})
cf_piva_clienti['Studio Affidatario'] = cf_piva_clienti['Studio Affidatario'].str.lower()
input_24 = pd.merge(input_24, cf_piva_clienti, left_on='Cliente',right_on='Studio Affidatario', how='left')
input_24.drop(columns=['Studio Affidatario'], inplace=True)

In [1]:
# creare colonne nome e cognome da debitore con queste regole:
# cognome è il cliente (denominazione completa)    ? 
# nome è il cognome del debitore 
#   se il cognome è 'de la cruz' usiamo il  copdice sotto per prendere almeno 'de la' 

def split_name(debitore):
    words = debitore.split()
    if len(words[0]) <=5:
        nome = ' '.join(words[0:2])
        if len(nome) <=5 : 
            nome = ' '.join(words[0:3])
    else:
        nome = words[0]
    return nome

# Apply the function to the 'debitore' column
input_24['nome'] = input_24['debitore'].apply(split_name)

SyntaxError: invalid syntax (2999151952.py, line 11)

In [22]:
def clean_string(s):
    replacements = {
        "'": " ",
        "è": "e",
        "é": "e",
        "à": "a",
        "ì": "i",
        "ò": "o",
        "ù": "u"
    }
    for old, new in replacements.items():
        s = s.replace(old, new)
    s = re.sub(' +', ' ', s)    
    s = s.strip()
    return s

input_24['debitore'] = input_24['debitore'].apply(clean_string)
input_24['Cliente'] = input_24['Cliente'].apply(clean_string)

,contratto,debitore,ufficio,atto,emesso da,repertorio,attore,convenuto,stato,compila,natura,tipo,numero,anno,Cliente,INTESTAZONE F23 PAGAMENTO,CF,nome,cognome
0,TBCO0012388330.074,LANZO ANTONIO FERNANDO,TARANTO UT DPTA,DECRETO INGIUNTIVO n. 07 del 2021,GIUDICE DI PACE DI S. GIORGIO,65/23 - 2023,TIBERIUS SPV S.R.L.,LANZO,IN ATTESA DI PAGAMENTO,si,DI - DECRETO INGIUNTIVO,DECRETO,07,2021,tiberius,Tiberius SPV Srl,04662720269,-,-
1,TBCO0012956551.074,FACCIO DIEGO,VICENZA UT DPVI,ESECUZIONE MOBILIARE n. 1162 del 2023,TRIBUNALE DI VICENZA,48 - 2024,TIBERIUS SPV S.R.L.,FACCIO,IN ATTESA DI PAGAMENTO,si,EM - ESECUZIONE MOBILIARE,PPT,1162,2023,tiberius,Tiberius SPV Srl,04662720269,DIEGO,FACCIO
2,TBS2037576,SCIVOLETTO ANTONINA,VERONA 2 UT DPVR,ESECUZIONE MOBILIARE n. 2099 del 2023,TRIBUNALE DI VERONA,211 - 2024,TIBERIUS SPV S.R.L.,SCIVOLETTO,IN ATTESA DI PAGAMENTO,si,EM - ESECUZIONE MOBILIARE,PPT,2099,2023,tiberius,Tiberius SPV Srl,04662720269,ANTONINA,SCIVOLETTO
3,TBS24010855,WANG PENGFEI,TORINO 1 UT DPTO1,ESECUZIONE MOBILIARE n. 4529 del 2023,TRIBUNALE TORINO,1001 - 2024,TIBERIUS SPV S.R.L.,WANG,IN ATTESA DI PAGAMENTO,si,EM - ESECUZIONE MOBILIARE,PPT,4529,2023,tiberius,Tiberius SPV Srl,04662720269,PENGFEI,WANG
4,TBS24682111,KOUAKEB SELLAMI,BOLOGNA 2 UT DPBO,ESECUZIONE MOBILIARE n. 959 del 2023,TRIBUNALE,380/24 - 2024,TIBERIUS SPV S.R.L.,KOUAKEB SELLAMI,IN ATTESA DI PAGAMENTO,si,EM - ESECUZIONE MOBILIARE,PPT,959,2023,tiberius,Tiberius SPV Srl,04662720269,SELLAMI,KOUAKEB
5,TBS27898708,HAPAN PEDIGE PRAGEETH TUSITHA VIJAYALATH,VERONA 2 UT DPVR,ESECUZIONE MOBILIARE n. 373 del 2024,TRIBUNALE DI VERONA,534/24 - 2024,TIBERIUS SPV S.R.L.,HAPAN PEDIGE,IN ATTESA DI PAGAMENTO,si,EM - ESECUZIONE MOBILIARE,PPT,373,2024,tiberius,Tiberius SPV Srl,04662720269,PRAGEETH TUSITHA VIJAYALATH,HAPAN PEDIGE
6,TBS52364746,PEPKOLAJ MIRJETA,GENOVA 1 UT DPGE,ESECUZIONE MOBILIARE n. 2805 del 2023,TRIBUNALE DI GENOVA,B640 - 2024,TIBERIUS SPV S.R.L.,PEPKOLAJ,IN ATTESA DI PAGAMENTO,si,EM - ESECUZIONE MOBILIARE,PPT,2805,2023,tiberius,Tiberius SPV Srl,04662720269,MIRJETA,PEPKOLAJ
7,TBS52495632,MD MOSTAFA,CREMONA UT DPCR,ESECUZIONE MOBILIARE n. 482 del 2023,TRIBUNALE DI CREMONA,519 - 2023,TIBERIUS SPV S.R.L.,MD,IN ATTESA DI PAGAMENTO,si,EM - ESECUZIONE MOBILIARE,PPT,482,2023,tiberius,Tiberius SPV Srl,04662720269,MOSTAFA,MD
8,TBS53001679,KALUGAMAGE ANTON SUJITH FERNANDO,TREVISO UT DPTV,DECRETO INGIUNTIVO n. 2830 del 2022,TRIBUNALE TREVISO,3852/23 - 2022,BLADE MANAGEMENT S.R.L.,KALUGAMAGE ANTON SUJITH FERNAN,IN ATTESA DI PAGAMENTO,si,DI - DECRETO INGIUNTIVO,DECRETO,2830,2022,tiberius,Tiberius SPV Srl,04662720269,SUJITH FERNANDO,KALUGAMAGE ANTON
9,TBS63943471,SCOGNAMIGLIO GIUSEPPE,NAPOLI 3 UT DPNA2,DECRETO INGIUNTIVO n. 158 del 2021,TRIBUNALE DI NAPOLI,4285 - 2021,BLADE MANAGEMENT S.R.L.,SCOGNAMIGLIO,IN ATTESA DI PAGAMENTO,si,DI - DECRETO INGIUNTIVO,DECRETO,158,2021,tiberius,Tiberius SPV Srl,04662720269,GIUSEPPE,SCOGNAMIGLIO


In [23]:
def click_button(driver, button):
    try:
        button.click()
    except Exception as e:
        # If clicking using regular method fails, try clicking using JavaScript
        driver.execute_script("arguments[0].click();", button)

def save_dict_to_json(dictionary, path):
    with open(path, 'w') as file:
        json.dump(dictionary, file)  

In [24]:
#   Codice per compilare F24 con piva nome e cognome


def select_option(driver, dropdown_locator, option_text):
    try:
        dropdown_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(dropdown_locator))
        dropdown_element.click()
        
        option_locator = (By.XPATH, f"//option[contains(text(), '{option_text}')]")
        option_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(option_locator))
        option_element.click()
    except Exception as e:
        print(f"Error: {e}")
        raise
       

def compila(driver,office_name,ente,anno,natura,numero,cf,cognome,nome,tipo):
    url = 'https://www1.agenziaentrate.gov.it/servizi/tassazioneattigiudiziari/registrazione.htm?passo=0'
    driver.get(url)

    try:
        #  inserire l'ufficio
        select_option(driver, (By.CSS_SELECTOR, "select#ufficio.form-control"), office_name)
        #clicca avanti
        select_locator = (By.CSS_SELECTOR, "input#avanti.btn.btn-primary")
        select_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(select_locator))
        click_button(driver, select_element)
        time.sleep(1)

        try:
            # clicca per inserire l'ente
            select_option(driver, (By.CSS_SELECTOR, "select#ente.form-control"), ente)
            # inserire l'anno
            select_option(driver, (By.CSS_SELECTOR, "select#anno.form-control"), anno)
            # inserire la natura
            select_option(driver, (By.CSS_SELECTOR, "select#natura.form-control"), natura)


            dropdown_element= WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "select#natura.form-control")))
            driver.execute_script("arguments[0].blur();", dropdown_element )
            
            # clicca per inserire il numero
            select_locator = (By.CSS_SELECTOR, "input#numero.form-control")
            select_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(select_locator))
            # driver.execute_script("arguments[0].scrollIntoView(true);", select_element)
            driver.execute_script("arguments[0].value = arguments[1];", select_element, numero)
                   
            time.sleep(2)

            #clicca avanti
            select_locator = (By.CSS_SELECTOR, "input#avanti.btn.btn-primary")
            select_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(select_locator))
            click_button(driver, select_element)
            time.sleep(2)

            # clicca su compila f24
            compila_f24_locator = (By.XPATH, "//a[contains(text(), 'Compila F24')]")
            compila_f24_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(compila_f24_locator))
            compila_f24_element.click()

            time.sleep(2)

            # clicca per inserire il codice fiscale
            select_locator = (By.CSS_SELECTOR, "input#codicefiscale")
            select_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(select_locator))
            # driver.execute_script("arguments[0].scrollIntoView(true);", select_element)
            driver.execute_script("arguments[0].value = arguments[1];", select_element, cf)

            # clicca per inserire il cognome
            select_locator = (By.CSS_SELECTOR, "input#cognome")
            select_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(select_locator))
            # driver.execute_script("arguments[0].scrollIntoView(true);", select_element)
            driver.execute_script("arguments[0].value = arguments[1];", select_element, cognome)

            # clicca per inserire il nome
            select_locator = (By.CSS_SELECTOR, "input#nome")
            select_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(select_locator))
            # driver.execute_script("arguments[0].scrollIntoView(true);", select_element)
            driver.execute_script("arguments[0].value = arguments[1];", select_element, nome)

            time.sleep(1)

            download_dir = os.path.join(os.path.expanduser("~"), "Downloads")
            existing_files = set(glob.glob(os.path.join(download_dir, "*")))

            downloaded_file_path = None
            timeout = 30  # Maximum time to wait for download in seconds
            start_time = time.time()

            stampaf24_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "stampaf24button")))
            stampaf24_button.click()

            time.sleep(3)

            while True:
                # Look for new files in the download directory
                new_files = set(glob.glob(os.path.join(download_dir, "*"))) - existing_files
                if new_files:
                    # Get the most recent file that does not have a .crdownload extension
                    completed_files = [f for f in new_files if not f.endswith('.crdownload')]
                    if completed_files:
                        downloaded_file_path = max(completed_files, key=os.path.getctime)
                        break
                if time.time() - start_time > timeout:
                    print("Timeout: File download took too long.")
                    break
                time.sleep(1)

            if downloaded_file_path:
                new_file_name = f"F24_{nome}_{tipo}.pdf"
                target_dir = f'F24_Download/{today}/Con_Nome'
                os.makedirs(target_dir, exist_ok=True)
                new_file_path = os.path.join(target_dir, new_file_name)
                os.rename(downloaded_file_path, new_file_path)
                print("File downloaded and moved to:", new_file_path)
            else:
                print("No files were downloaded.")
                
        except Exception as e:
            print(f"Error: {e}")
            return []   

    except Exception as e:
        print(f"Error: {e}")
        return []



     

def main(row):
    options = webdriver.EdgeOptions()
    #options.add_argument("--headless")
    driver = webdriver.Edge(options=options)
    driver.maximize_window()
    
    # compila(driver,office_name='TARANTO  UT DPTA',ente='GIUDICE DI PACE DI S. GIORGIO',anno='2021',natura='DI - DECRETO INGIUNTIVO',numero='07',cf='04662720269',cognome='Lanzo',nome='Antonio Fernando')
    compila(
        driver=driver, 
        office_name=row['ufficio'],
        ente=row['emesso da'],
        anno=row['anno'],
        natura=row['natura'],
        numero=row['numero'],
        cf=row['CF'],
        cognome=row['INTESTAZONE F23 PAGAMENTO'],
        nome=row['nome'],
        tipo=row['tipo']
    ) 
    driver.quit()

 
if __name__ == "__main__":
    filtered_df = input_24[input_24['nome'] != '-']
    filtered_df.apply(main, axis=1)


File downloaded and moved to: F24_Download/2024-05-30/Con_Nome\F24_DIEGO_FACCIO_PPT.pdf
File downloaded and moved to: F24_Download/2024-05-30/Con_Nome\F24_ANTONINA_SCIVOLETTO_PPT.pdf
File downloaded and moved to: F24_Download/2024-05-30/Con_Nome\F24_PENGFEI_WANG_PPT.pdf
File downloaded and moved to: F24_Download/2024-05-30/Con_Nome\F24_SELLAMI_KOUAKEB_PPT.pdf
File downloaded and moved to: F24_Download/2024-05-30/Con_Nome\F24_PRAGEETH TUSITHA VIJAYALATH_HAPAN PEDIGE_PPT.pdf
File downloaded and moved to: F24_Download/2024-05-30/Con_Nome\F24_MIRJETA_PEPKOLAJ_PPT.pdf
File downloaded and moved to: F24_Download/2024-05-30/Con_Nome\F24_MOSTAFA_MD_PPT.pdf
File downloaded and moved to: F24_Download/2024-05-30/Con_Nome\F24_SUJITH FERNANDO_KALUGAMAGE ANTON_DECRETO.pdf
File downloaded and moved to: F24_Download/2024-05-30/Con_Nome\F24_GIUSEPPE_SCOGNAMIGLIO_DECRETO.pdf
File downloaded and moved to: F24_Download/2024-05-30/Con_Nome\F24_STEFANO_TERRACCIANO_DECRETO.pdf


In [25]:
#   Codice per compilare F24 solo con piva


def select_option(driver, dropdown_locator, option_text):
    try:
        dropdown_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(dropdown_locator))
        dropdown_element.click()
        
        option_locator = (By.XPATH, f"//option[contains(text(), '{option_text}')]")
        option_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(option_locator))
        option_element.click()
    except Exception as e:
        print(f"Error: {e}")
        raise
       

def compila(driver,office_name,ente,anno,natura,numero,cf,cognome,nome,tipo):
    url = 'https://www1.agenziaentrate.gov.it/servizi/tassazioneattigiudiziari/registrazione.htm?passo=0'
    driver.get(url)

    try:
        #  inserire l'ufficio
        select_option(driver, (By.CSS_SELECTOR, "select#ufficio.form-control"), office_name)
        #clicca avanti
        select_locator = (By.CSS_SELECTOR, "input#avanti.btn.btn-primary")
        select_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(select_locator))
        click_button(driver, select_element)
        time.sleep(1)

        try:
            # clicca per inserire l'ente
            select_option(driver, (By.CSS_SELECTOR, "select#ente.form-control"), ente)
            # inserire l'anno
            select_option(driver, (By.CSS_SELECTOR, "select#anno.form-control"), anno)
            # inserire la natura
            select_option(driver, (By.CSS_SELECTOR, "select#natura.form-control"), natura)


            dropdown_element= WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "select#natura.form-control")))
            driver.execute_script("arguments[0].blur();", dropdown_element )
            
            # clicca per inserire il numero
            select_locator = (By.CSS_SELECTOR, "input#numero.form-control")
            select_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(select_locator))
            # driver.execute_script("arguments[0].scrollIntoView(true);", select_element)
            driver.execute_script("arguments[0].value = arguments[1];", select_element, numero)
                   
            time.sleep(2)

            #clicca avanti
            select_locator = (By.CSS_SELECTOR, "input#avanti.btn.btn-primary")
            select_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(select_locator))
            click_button(driver, select_element)
            time.sleep(2)

            # clicca su compila f24
            compila_f24_locator = (By.XPATH, "//a[contains(text(), 'Compila F24')]")
            compila_f24_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(compila_f24_locator))
            compila_f24_element.click()

            time.sleep(2)

            # clicca per inserire il codice fiscale
            select_locator = (By.CSS_SELECTOR, "input#codicefiscale")
            select_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(select_locator))
            # driver.execute_script("arguments[0].scrollIntoView(true);", select_element)
            driver.execute_script("arguments[0].value = arguments[1];", select_element, cf)

            time.sleep(1)

            download_dir = os.path.join(os.path.expanduser("~"), "Downloads")
            existing_files = set(glob.glob(os.path.join(download_dir, "*")))

            downloaded_file_path = None
            timeout = 30  # Maximum time to wait for download in seconds
            start_time = time.time()

            stampaf24_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "stampaf24button")))
            stampaf24_button.click()

            time.sleep(3)

            while True:
                # Look for new files in the download directory
                new_files = set(glob.glob(os.path.join(download_dir, "*"))) - existing_files
                if new_files:
                    # Get the most recent file that does not have a .crdownload extension
                    completed_files = [f for f in new_files if not f.endswith('.crdownload')]
                    if completed_files:
                        downloaded_file_path = max(completed_files, key=os.path.getctime)
                        break
                if time.time() - start_time > timeout:
                    print("Timeout: File download took too long.")
                    break
                time.sleep(1)

            if downloaded_file_path:
                new_file_name = f"F24_{nome}_{tipo}.pdf"
                target_dir = f'F24_Download/{today}/Con_Piva'
                os.makedirs(target_dir, exist_ok=True)
                new_file_path = os.path.join(target_dir, new_file_name)
                os.rename(downloaded_file_path, new_file_path)
                print("File downloaded and moved to:", new_file_path)
            else:
                print("No files were downloaded.")
                
        except Exception as e:
            print(f"Error: {e}")
            return []   

    except Exception as e:
        print(f"Error: {e}")
        return []



     

def main(row):
    options = webdriver.EdgeOptions()
    #options.add_argument("--headless")
    driver = webdriver.Edge(options=options)
    driver.maximize_window()
    
    # compila(driver,office_name='TARANTO  UT DPTA',ente='GIUDICE DI PACE DI S. GIORGIO',anno='2021',natura='DI - DECRETO INGIUNTIVO',numero='07',cf='04662720269',cognome='Lanzo',nome='Antonio Fernando')
    compila(
        driver=driver, 
        office_name=row['ufficio'],
        ente=row['emesso da'],
        anno=row['anno'],
        natura=row['natura'],
        numero=row['numero'],
        cf=row['CF'],
        cognome=row['INTESTAZONE F23 PAGAMENTO'],
        nome=row['nome'],
        tipo=row['tipo']
    ) 
    driver.quit()

 
if __name__ == "__main__":
    filtered_df = input_24[input_24['nome'] != '-']
    filtered_df.apply(main, axis=1)


File downloaded and moved to: F24_Download/2024-05-30/Con_Piva\F24_DIEGO_FACCIO_PPT.pdf
File downloaded and moved to: F24_Download/2024-05-30/Con_Piva\F24_ANTONINA_SCIVOLETTO_PPT.pdf
File downloaded and moved to: F24_Download/2024-05-30/Con_Piva\F24_PENGFEI_WANG_PPT.pdf
File downloaded and moved to: F24_Download/2024-05-30/Con_Piva\F24_SELLAMI_KOUAKEB_PPT.pdf
File downloaded and moved to: F24_Download/2024-05-30/Con_Piva\F24_PRAGEETH TUSITHA VIJAYALATH_HAPAN PEDIGE_PPT.pdf
File downloaded and moved to: F24_Download/2024-05-30/Con_Piva\F24_MIRJETA_PEPKOLAJ_PPT.pdf
File downloaded and moved to: F24_Download/2024-05-30/Con_Piva\F24_MOSTAFA_MD_PPT.pdf
File downloaded and moved to: F24_Download/2024-05-30/Con_Piva\F24_SUJITH FERNANDO_KALUGAMAGE ANTON_DECRETO.pdf
File downloaded and moved to: F24_Download/2024-05-30/Con_Piva\F24_GIUSEPPE_SCOGNAMIGLIO_DECRETO.pdf
File downloaded and moved to: F24_Download/2024-05-30/Con_Piva\F24_STEFANO_TERRACCIANO_DECRETO.pdf
